In [ ]:
import scipy.io
data = scipy.io.loadmat('Data_ankle.mat')
ankle_data= data['ankle_data']
print(ankle_data.shape)

(48000, 7)


# New Section

In [ ]:
import pandas as pd
# Define column names
column_names = [
    'time',
    'vgrf',  # vertical ground reaction force
    'trunk_angle',
    'hip_angle',
    'knee_angle',
    'ankle_angle',
    'joint_moment'
]


In [ ]:
df = pd.DataFrame(ankle_data, columns=column_names )
df.head()

,time,vgrf,trunk_angle,hip_angle,knee_angle,ankle_angle,joint_moment
0,0.00,-10.321381,-0.094286,0.292974,0.825063,-1.492588,-2.011628
1,0.01,-7.990312,-0.096174,0.293179,0.822469,-1.493192,-3.655209
2,0.02,-6.208738,-0.096449,0.294008,0.810997,-1.495867,-5.786433
3,0.03,-5.126637,-0.097196,0.295568,0.785731,-1.501793,-6.190008
4,0.04,-4.774037,-0.095624,0.297472,0.745707,-1.511208,-5.542868


In [ ]:
# Split the data into training and testing sets
train_size = int(0.8 * len(df))
train_data = df[:train_size]
test_data = df[train_size:]

In [ ]:
# Separate the features and target variable
X_train = train_data.drop(columns=['time', 'joint_moment'])
Y_train = train_data['joint_moment'].values.reshape(-1, 1)  # Reshape for scaler
X_test = test_data.drop(columns=['time', 'joint_moment'])
Y_test = test_data['joint_moment'].values.reshape(-1, 1)  # Reshape for scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
# Apply scaling only to the training data
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)  # Use the same scaler to transform the test data

In [ ]:
# Apply scaling to the target variable
scaler_Y = StandardScaler()
Y_train_scaled = scaler_Y.fit_transform(Y_train)  # Fit and transform the training target variable
Y_test_scaled = scaler_Y.transform(Y_test)  # Transform the test target variable using the same scaler

print(Y_test_scaled)

[[-0.98351688]
 [-0.98411872]
 [-0.96365408]
 ...
 [ 0.30780874]
 [ 0.34680355]
 [ 0.37615492]]


In [ ]:
# Convert scaled data back to DataFrame
normalized_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
normalized_train_df['time'] = train_data['time'].reset_index(drop=True)
normalized_train_df['joint_moment'] = Y_train_scaled.flatten()  # Flatten to 1D

normalized_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
normalized_test_df['time'] = test_data['time'].reset_index(drop=True)
normalized_test_df['joint_moment'] = Y_test_scaled.flatten()  # Flatten to 1D

In [ ]:
# Print the normalized data
print(train_data.head())
print(test_data.head())

# If you need to access the target variable from the test data separately
target_Ytest_normalized = normalized_test_df['joint_moment']
print(target_Ytest_normalized.head())

   time       vgrf  trunk_angle  hip_angle  knee_angle  ankle_angle  \
0  0.00 -10.321381    -0.094286   0.292974    0.825063    -1.492588   
1  0.01  -7.990312    -0.096174   0.293179    0.822469    -1.493192   
2  0.02  -6.208738    -0.096449   0.294008    0.810997    -1.495867   
3  0.03  -5.126637    -0.097196   0.295568    0.785731    -1.501793   
4  0.04  -4.774037    -0.095624   0.297472    0.745707    -1.511208   

   joint_moment  
0     -2.011628  
1     -3.655209  
2     -5.786433  
3     -6.190008  
4     -5.542868  
         time       vgrf  trunk_angle  hip_angle  knee_angle  ankle_angle  \
38400  384.00 -11.512108    -0.032512   0.007387    0.973938    -1.273562   
38401  384.01 -15.839764    -0.035124   0.037376    1.022451    -1.272575   
38402  384.02 -16.915961    -0.038637   0.066333    1.062992    -1.280968   
38403  384.03 -15.981315    -0.044385   0.093862    1.094613    -1.295619   
38404  384.04 -13.927180    -0.048895   0.119811    1.116854    -1.313351   

  

In [ ]:
def extract_data_for_combination(columns):
    X_train = normalized_train_df[columns].values
    y_train = normalized_train_df['joint_moment'].values
    X_test = normalized_test_df[columns].values
    y_test = normalized_test_df['joint_moment'].values

    # Reshape for RNN
    X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

    # Reshape for CNN
    X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    return X_train_rnn, y_train, X_test_rnn, y_test, X_train_cnn, X_test_cnn


In [ ]:
from keras.models import Sequential
from keras.layers import  Dense, Dropout , LSTM ,  BatchNormalization

# RNN Model creation function
def create_rnn_model(input_shape, units, dropout_rate):
    model = Sequential()

    # First LSTM layer
    model.add(LSTM(units, activation='relu', return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))

    # Second LSTM layer
    model.add(LSTM(units, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Batch Normalization
    model.add(BatchNormalization())

    # Dense layer
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Flatten, Dropout, BatchNormalization

# CNN Model creation function
def create_cnn_model(input_shape, units, dropout_rate):
    model = Sequential()

    # First Conv1D layer with kernel size 1
    model.add(Conv1D(filters=units, kernel_size=1, activation='relu', input_shape=input_shape))
    model.add(Dropout(dropout_rate))

    # Second Conv1D layer with kernel size 1
    model.add(Conv1D(filters=units, kernel_size=1, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Flatten layer
    model.add(Flatten())

    # Batch Normalization
    model.add(BatchNormalization())

    # Dense layer
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse')
    return model



In [ ]:
from sklearn.metrics import mean_squared_error
import os

def train_and_evaluate_rnn_model(input_comb, model_path):
    X_train_rnn, y_train, X_test_rnn, y_test, _, _ = extract_data_for_combination(input_comb)
    rnn_model = create_rnn_model(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]))
    history = rnn_model.fit(X_train_rnn, y_train, epochs=53, batch_size=64, validation_split=0.2, verbose=1)
    predictions = rnn_model.predict(X_test_rnn).flatten()
    mse = mean_squared_error(y_test, predictions)
    rnn_model.save(model_path)
    return mse, predictions, history

In [ ]:
def train_and_evaluate_cnn_model(input_comb, model_path):
    _, y_train, _, y_test, X_train_cnn, X_test_cnn = extract_data_for_combination(input_comb)
    cnn_model = create_cnn_model(input_shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]))
    history = cnn_model.fit(X_train_cnn, y_train, epochs=50, batch_size=64, validation_split=0.2, verbose=1)
    predictions = cnn_model.predict(X_test_cnn).flatten()
    mse = mean_squared_error(y_test, predictions)
    cnn_model.save(model_path)
    return mse, predictions, history

In [ ]:
import os
import optuna
import optuna.visualization as vis
import matplotlib.pyplot as plt


In [32]:
def objective(trial, model_type, input_comb):
    # Suggest hyperparameters for both RNN and CNN
    units = trial.suggest_int('units', 10, 100)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    epochs = trial.suggest_int('epochs', 40, 60)
    batch_size = trial.suggest_int('batch_size', 16, 128)

    # Extract data for the input combination
    X_train_rnn, y_train, X_test_rnn, y_test, X_train_cnn, X_test_cnn = extract_data_for_combination(input_comb)

    if model_type == 'rnn':
        model = create_rnn_model(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), units=units, dropout_rate=dropout_rate)
        history = model.fit(X_train_rnn, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
        predictions = model.predict(X_test_rnn).flatten()

    elif model_type == 'cnn':
        model = create_cnn_model(input_shape=(X_train_cnn.shape[1], X_train_cnn.shape[2]), units=units, dropout_rate=dropout_rate)
        history = model.fit(X_train_cnn, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
        predictions = model.predict(X_test_cnn).flatten()

    mse = mean_squared_error(y_test, predictions)
    return mse


In [35]:
# Function to optimize model using Optuna
def optimize_model(model_type, input_comb_key, input_comb, n_trials=12):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, model_type, input_comb), n_trials=n_trials)

    best_trial = study.best_trial
    print(f"Best trial for {model_type} with input {input_comb_key}:")
    print(f"  Value: {best_trial.value}")
    print(f"  Params: ")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")

    # Generate and display Optuna visualizations
    #vis.plot_optimization_history(study).show()
    #vis.plot_param_importances(study).show()

    return best_trial


In [36]:
# Define input combinations
input_combinations = {
    'all_angles_and_vgrf': ['vgrf', 'trunk_angle', 'hip_angle', 'knee_angle', 'ankle_angle'],
    # 'ankle_angle_only': ['vgrf', 'ankle_angle'],
    # 'all_angles': ['trunk_angle', 'hip_angle', 'knee_angle', 'ankle_angle'],
}

results = {}
for key, columns in input_combinations.items():
    print(f"Optimizing RNN for input combination: {key}")
    rnn_best_trial = optimize_model('rnn', key, columns, n_trials=12)

    print(f"Optimizing CNN for input combination: {key}")
    cnn_best_trial = optimize_model('cnn', key, columns, n_trials=12)

    results[key] = {
        'rnn_best_trial': rnn_best_trial,
        'cnn_best_trial': cnn_best_trial
    }

print(results)

[I 2024-07-29 01:48:18,727] A new study created in memory with name: no-name-0c1cebe4-dddf-4be2-9b29-a11f2108de12


Optimizing RNN for input combination: all_angles_and_vgrf
300/300 [==============================] - 0s 428us/step


[I 2024-07-29 01:48:51,063] Trial 0 finished with value: 0.009570368489226084 and parameters: {'units': 46, 'dropout_rate': 0.49171347746635075, 'epochs': 54, 'batch_size': 118}. Best is trial 0 with value: 0.009570368489226084.


300/300 [==============================] - 0s 546us/step


[I 2024-07-29 01:49:44,801] Trial 1 finished with value: 0.006540652690838576 and parameters: {'units': 87, 'dropout_rate': 0.2863806338505511, 'epochs': 54, 'batch_size': 80}. Best is trial 1 with value: 0.006540652690838576.


...

300/300 [==============================] - 0s 786us/step


[I 2024-07-29 02:09:01,768] Trial 10 finished with value: 0.010281003882098823 and parameters: {'units': 99, 'dropout_rate': 0.10763673140511917, 'epochs': 40, 'batch_size': 95}. Best is trial 10 with value: 0.010281003882098823.


300/300 [==============================] - 0s 404us/step


[I 2024-07-29 02:09:33,592] Trial 11 finished with value: 0.01139527884882943 and parameters: {'units': 98, 'dropout_rate': 0.12101539920928214, 'epochs': 40, 'batch_size': 94}. Best is trial 10 with value: 0.010281003882098823.


Best trial for cnn with input all_angles_and_vgrf:
  Value: 0.010281003882098823
  Params: 
    units: 99
    dropout_rate: 0.10763673140511917
    epochs: 40
    batch_size: 95
{'all_angles_and_vgrf': {'rnn_best_trial': FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.006540652690838576], datetime_start=datetime.datetime(2024, 7, 29, 1, 48, 51, 63732), datetime_complete=datetime.datetime(2024, 7, 29, 1, 49, 44, 801674), params={'units': 87, 'dropout_rate': 0.2863806338505511, 'epochs': 54, 'batch_size': 80}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'units': IntDistribution(high=100, log=False, low=10, step=1), 'dropout_rate': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'epochs': IntDistribution(high=60, log=False, low=40, step=1), 'batch_size': IntDistribution(high=128, log=False, low=16, step=1)}, trial_id=1, value=None), 'cnn_best_trial': FrozenTrial(number=10, state=TrialState.COMPLETE, values=[0.010281003882098823], datetime_

In [37]:
print(results)
 

{'all_angles_and_vgrf': {'rnn_best_trial': FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.006540652690838576], datetime_start=datetime.datetime(2024, 7, 29, 1, 48, 51, 63732), datetime_complete=datetime.datetime(2024, 7, 29, 1, 49, 44, 801674), params={'units': 87, 'dropout_rate': 0.2863806338505511, 'epochs': 54, 'batch_size': 80}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'units': IntDistribution(high=100, log=False, low=10, step=1), 'dropout_rate': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'epochs': IntDistribution(high=60, log=False, low=40, step=1), 'batch_size': IntDistribution(high=128, log=False, low=16, step=1)}, trial_id=1, value=None), 'cnn_best_trial': FrozenTrial(number=10, state=TrialState.COMPLETE, values=[0.010281003882098823], datetime_start=datetime.datetime(2024, 7, 29, 2, 8, 20, 574865), datetime_complete=datetime.datetime(2024, 7, 29, 2, 9, 1, 768004), params={'units': 99, 'dropout_rate': 0.1076367314051191